# Summarize Pharmas
This script creates an overview over the data

In [1]:
import pandas as pd
import sys
sys.path.insert(0, '../../data/lib/')
import consts
from pdfexport import *
import openpyxl

In [2]:
ACCUM_IN = '../../data/3. transformation/1_accumulations_all.csv'
FILE_OUT = '../../data/99. analyzes/excel/'

#exclude_list = ['novartis', 'takeda', 'menarini']
exclude_list = []

In [3]:
#Load Data
df_hcp = pd.read_csv('../../data/3. transformation/6_hcp_grouped.csv')
df_hco = pd.read_csv('../../data/3. transformation/6_hco_grouped.csv')

df_raw_data = pd.concat([df_hcp, df_hco])

df_raw_accumulations = pd.read_csv(ACCUM_IN)

/Users/hws/Documents/python/pharmagelder/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [4]:
def run(year, filename):
    global df_raw_data
    global df_raw_accumulations
    global FILE_OUT
    global exclude_list

    if year == 0:
        df_list = df_raw_data.copy()
        df_accumulations = df_raw_accumulations.copy()        
    else:
        df_list = df_raw_data[df_raw_data.year == year].copy()
        df_accumulations = df_raw_accumulations[df_raw_accumulations.year == year].copy()
    
    #Remove files, which we have not yet properly cleaned
    for excl in exclude_list:
        df_list = df_list[df_list.source != excl]
        df_accumulations = df_accumulations[df_accumulations.source != excl]

    #Numberize
    df_list = amounts_to_number(df_list)
    df_accumulations = amounts_to_number(df_accumulations)


    #Recalculate Total - do not take it frome the total field. sum it
    df_list = df_list.fillna(0)
    df_list['total'] = df_list.apply(lambda row: (float(row['donations_grants'])
    + float(row['sponsorship'])
    + float(row['registration_fees'])
    + float(row['travel_accommodation'])
    + float(row['fees'])
    + float(row['related_expenses'])), axis=1)

    df_accumulations = df_accumulations.fillna(0)
    df_accumulations.loc[df_accumulations['type'].isin(['hcp_amount', 'hco_amount']), 'total'] = df_accumulations.apply(lambda row: (float(row['donations_grants'])
    + float(row['sponsorship'])
    + float(row['registration_fees'])
    + float(row['travel_accommodation'])
    + float(row['fees'])
    + float(row['related_expenses'])), axis=1)    
    
    df_accumulations_detailed = df_accumulations.copy()
    df_accumulations_detailed = df_accumulations_detailed[~df_accumulations_detailed.type.isin(['hco_count', 'hcp_count'])]

    #Made Simple Accumulations
    df_accumulations = df_accumulations.replace("hcp_amount", 'hcp')
    df_accumulations = df_accumulations.replace("hco_amount", 'hco')
    df_accumulations = df_accumulations[~df_accumulations.type.isin(['hco_count', 'hcp_count'])]
    
    
    #Concat
    df_concat_simple = pd.concat([df_list, df_accumulations], sort=True)
    df_concat_detailed = pd.concat([df_list, df_accumulations_detailed], sort=True)

    #Group
    df_grouped_sum = df_concat_simple.groupby(by=['source']).sum()

    df_grouped_sum['rnd'] = df_grouped_sum['total'] - (df_grouped_sum['donations_grants']
    + df_grouped_sum['sponsorship'] + df_grouped_sum['registration_fees'] + df_grouped_sum['travel_accommodation']
    + df_grouped_sum['fees'] + df_grouped_sum['related_expenses'])
    
    df_grouped_simple = df_concat_simple.groupby(by=['source', 'type']).sum()
    df_grouped_detail = df_concat_detailed.groupby(by=['source', 'type']).sum()

    #Offen vs. Anonym
    df_offen = df_list.groupby(by='source').agg({'total': 'sum'})
    df_anonym = df_accumulations[df_accumulations.type.isin(['hcp', 'hco'])]
    df_anonym = df_anonym.groupby('source').agg({'total': 'sum'})
    df_offen['type'] = 'offen'
    df_anonym['type'] = 'anonym'
    df_anonymoffen = pd.concat([df_offen, df_anonym])
    df_anonymoffen = df_anonymoffen.reset_index()
    df_pivat_anonymoffen = df_anonymoffen.pivot(index='source', columns='type', values='total')
    df_pivat_anonymoffen = df_pivat_anonymoffen.fillna(0)
    df_pivat_anonymoffen['total'] = df_pivat_anonymoffen['anonym'] + df_pivat_anonymoffen['offen']
    df_pivat_anonymoffen['offen_percentage'] = df_pivat_anonymoffen.apply(lambda row: round(100 / row['total'] * row['offen']), axis=1)
    df_pivat_anonymoffen['anonym_percentage'] = df_pivat_anonymoffen.apply(lambda row: round(100 / row['total'] * row['anonym']), axis=1)
    df_pivat_anonymoffen = df_pivat_anonymoffen[['offen', 'offen_percentage', 'anonym', 'anonym_percentage', 'total']]
    df_pivat_anonymoffen = df_pivat_anonymoffen.sort_values('offen_percentage', ascending=True)

    #Group
    df_grouped_sum = df_grouped_sum[['donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'rnd', 'total']]
    df_grouped_simple = df_grouped_simple[['donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'total']]
    df_grouped_detail = df_grouped_detail[['donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'total']]

    #Save to Excel
    writer = pd.ExcelWriter(FILE_OUT + filename, options={'encoding':'utf-8'})

    # Write the frame to excel
    df_info = pd.DataFrame({'info': ['Tabellen (Tabs) beachten!']})
    df_info.to_excel(writer, 'info', index=False)
    df_grouped_sum.to_excel(writer, 'total', index=True)
    df_grouped_simple.to_excel(writer, 'detail', index=True)
    df_grouped_detail.to_excel(writer, 'detail_nach_offen_anonym', index=True)
    df_pivat_anonymoffen.to_excel(writer, 'offen_vs_anonym', index=True)

    writer.save()
    print("%s finished" % year)
    
run(2017, '2017_pharma_summarize.xlsx')
run(2016, '2016_pharma_summarize.xlsx')
run(2015, '2015_pharma_summarize.xlsx')
run(0, 'all_pharma_summarize.xlsx')


/Users/hws/Documents/python/pharmagelder/env/lib/python3.6/site-packages/ipykernel_launcher.py:75: RuntimeWarning: divide by zero encountered in double_scalars
/Users/hws/Documents/python/pharmagelder/env/lib/python3.6/site-packages/ipykernel_launcher.py:75: RuntimeWarning: invalid value encountered in double_scalars
/Users/hws/Documents/python/pharmagelder/env/lib/python3.6/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in double_scalars
/Users/hws/Documents/python/pharmagelder/env/lib/python3.6/site-packages/ipykernel_launcher.py:76: RuntimeWarning: invalid value encountered in double_scalars


2017 finished
2016 finished
2015 finished
0 finished
